In [22]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation,Dense,Conv2D,MaxPool2D,Flatten,BatchNormalization,Reshape,InputLayer,GlobalAveragePooling2D,DepthwiseConv2D,Dropout,MaxPooling2D,ZeroPadding2D
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

import itertools

In [2]:
import os
os.chdir('Air Dataset')

In [3]:
train_path='train'
test_path='test'
valid_path='valid'

In [4]:
train_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=train_path,target_size=(512,512),batch_size=32)
valid_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=valid_path,target_size=(512,512),batch_size=4)
test_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=test_path,target_size=(512,512),batch_size=4,shuffle=False)


Found 2244 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.


In [5]:
mobile=tf.keras.applications.mobilenet.MobileNet()


MobileNet v1Lite without Bitshift

In [14]:
model=Sequential()
model.add(InputLayer(shape=(512, 512, 3)))

In [15]:
for layer in mobile.layers[1:43]:
     if isinstance(layer, Conv2D):
        
        #print(f'Layer {layer.name} is a Conv2D layer with {layer.filters} filters')
        newfilters=(int)(layer.filters/4)
        newlayer=Conv2D(newfilters, layer.kernel_size, strides=layer.strides,
                      padding=layer.padding, activation=layer.activation,
                      use_bias=layer.use_bias, kernel_initializer=layer.kernel_initializer,
                      bias_initializer=layer.bias_initializer)
        model.add(newlayer)
        print(f'Layer {newlayer.name} is a Conv2D layer with {newlayer.filters} filters and {newlayer.strides} strides and {newlayer.kernel_size} kernel_size')
     elif isinstance(layer, BatchNormalization) :
        model.add(BatchNormalization(axis=getattr(layer, 'axis', -1),
    momentum=getattr(layer, 'momentum', 0.99),
    epsilon=getattr(layer, 'epsilon', 1e-3),
    center=getattr(layer, 'center', True),
    scale=getattr(layer, 'scale', True),
    beta_initializer=getattr(layer, 'beta_initializer', 'zeros'),
    gamma_initializer=getattr(layer, 'gamma_initializer', 'ones'),
    moving_mean_initializer=getattr(layer, 'moving_mean_initializer', 'zeros'),
    moving_variance_initializer=getattr(layer, 'moving_variance_initializer', 'ones'),
    beta_regularizer=getattr(layer, 'beta_regularizer', None),
    gamma_regularizer=getattr(layer, 'gamma_regularizer', None),
    beta_constraint=getattr(layer, 'beta_constraint', None),
    gamma_constraint=getattr(layer, 'gamma_constraint', None)))
     elif isinstance(layer,DepthwiseConv2D):
         print(f'Layer {layer.name} is a DepthConv2D layer with  {layer.strides} strides and {layer.kernel_size} kernel size')
         model.add(DepthwiseConv2D(kernel_size=getattr(layer, 'kernel_size', (3, 3)),
    strides=getattr(layer, 'strides', (1, 1)),
    padding=getattr(layer, 'padding', 'valid'),
    depth_multiplier=getattr(layer, 'depth_multiplier', 1),
    activation=getattr(layer, 'activation', None),
    use_bias=getattr(layer, 'use_bias', True),
    bias_initializer=getattr(layer, 'bias_initializer', 'zeros'),
    depthwise_initializer=getattr(layer, 'depthwise_initializer', 'glorot_uniform')
))
     #elif(isinstance(layer,DepthwiseConv2D)):
        #     print('hellooooooooooo')
       #      print(f'Layer {layer.name} is a DepthConv2D layer with  {layer.strides} strides')
             #model.add(layer)
     elif (isinstance(layer,ZeroPadding2D)):
         print(1)

     else:
          #print(layer.name)
          model.add(layer)
     
   

Layer conv2d_35 is a Conv2D layer with 8 filters and (2, 2) strides and (3, 3) kernel_size
Layer conv_dw_1 is a DepthConv2D layer with  (1, 1) strides and (3, 3) kernel size
Layer conv2d_36 is a Conv2D layer with 16 filters and (1, 1) strides and (1, 1) kernel_size
1
Layer conv_dw_2 is a DepthConv2D layer with  (2, 2) strides and (3, 3) kernel size
Layer conv2d_37 is a Conv2D layer with 32 filters and (1, 1) strides and (1, 1) kernel_size
Layer conv_dw_3 is a DepthConv2D layer with  (1, 1) strides and (3, 3) kernel size
Layer conv2d_38 is a Conv2D layer with 32 filters and (1, 1) strides and (1, 1) kernel_size
1
Layer conv_dw_4 is a DepthConv2D layer with  (2, 2) strides and (3, 3) kernel size
Layer conv2d_39 is a Conv2D layer with 64 filters and (1, 1) strides and (1, 1) kernel_size
Layer conv_dw_5 is a DepthConv2D layer with  (1, 1) strides and (3, 3) kernel size
Layer conv2d_40 is a Conv2D layer with 64 filters and (1, 1) strides and (1, 1) kernel_size
1
Layer conv_dw_6 is a DepthCo

In [16]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_35 (Conv2D)              │ (None, 256, 256, 8)    │           216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_65          │ (None, 256, 256, 8)    │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 256, 256, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_30             │ (None, 256, 256, 8)    │            72 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_66          │ (None, 256, 256, 8)    │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 256, 256, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ (None, 256, 256, 16)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_67          │ (None, 256, 256, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 256, 256, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_31             │ (None, 127, 127, 16)   │           144 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_68          │ (None, 127, 127, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 127, 127, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_37 (Conv2D)              │ (None, 127, 127, 32)   │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_69          │ (None, 127, 127, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_32             │ (None, 127, 127, 32)   │           288 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_70          │ (None, 127, 127, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_38 (Conv2D)              │ (None, 127, 127, 32)   │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_71          │ (None, 127, 127, 32)   │           128 │
│ (BatchNormalization)            │                        │             

 Total params: 20,400 (79.69 KB)

 Trainable params: 19,280 (75.31 KB)

 Non-trainable params: 1,120 (4.38 KB)

In [17]:
for layer in mobile.layers[73:86]:
     if isinstance(layer, Conv2D):
        
        #print(f'Layer {layer.name} is a Conv2D layer with {layer.filters} filters')
        newfilters=(int)(layer.filters/4)
        newlayer=Conv2D(newfilters, layer.kernel_size, strides=layer.strides,
                      padding=layer.padding, activation=layer.activation,
                      use_bias=layer.use_bias, kernel_initializer=layer.kernel_initializer,
                      bias_initializer=layer.bias_initializer)
        model.add(newlayer)
        print(f'Layer {newlayer.name} is a Conv2D layer with {newlayer.filters} filters and {newlayer.strides} strides and {newlayer.kernel_size} kernel_size')
     elif isinstance(layer, BatchNormalization) :
        model.add(BatchNormalization(axis=getattr(layer, 'axis', -1),
    momentum=getattr(layer, 'momentum', 0.99),
    epsilon=getattr(layer, 'epsilon', 1e-3),
    center=getattr(layer, 'center', True),
    scale=getattr(layer, 'scale', True),
    beta_initializer=getattr(layer, 'beta_initializer', 'zeros'),
    gamma_initializer=getattr(layer, 'gamma_initializer', 'ones'),
    moving_mean_initializer=getattr(layer, 'moving_mean_initializer', 'zeros'),
    moving_variance_initializer=getattr(layer, 'moving_variance_initializer', 'ones'),
    beta_regularizer=getattr(layer, 'beta_regularizer', None),
    gamma_regularizer=getattr(layer, 'gamma_regularizer', None),
    beta_constraint=getattr(layer, 'beta_constraint', None),
    gamma_constraint=getattr(layer, 'gamma_constraint', None)))
     elif isinstance(layer,DepthwiseConv2D):
         print(f'Layer {layer.name} is a DepthConv2D layer with  {layer.strides} strides and {layer.kernel_size} kernel size')

         model.add(DepthwiseConv2D(kernel_size=getattr(layer, 'kernel_size', (3, 3)),
    strides=getattr(layer, 'strides', (1, 1)),
    padding=getattr(layer, 'padding', 'valid'),
    depth_multiplier=getattr(layer, 'depth_multiplier', 1),
    activation=getattr(layer, 'activation', None),
    use_bias=getattr(layer, 'use_bias', True),
    bias_initializer=getattr(layer, 'bias_initializer', 'zeros'),
    depthwise_initializer=getattr(layer, 'depthwise_initializer', 'glorot_uniform')
))
     elif (isinstance(layer,ZeroPadding2D)):
         print(1) 
     else:
         model.add(layer)

   

1
Layer conv_dw_12 is a DepthConv2D layer with  (2, 2) strides and (3, 3) kernel size
Layer conv2d_42 is a Conv2D layer with 256 filters and (1, 1) strides and (1, 1) kernel_size
Layer conv_dw_13 is a DepthConv2D layer with  (1, 1) strides and (3, 3) kernel size
Layer conv2d_43 is a Conv2D layer with 256 filters and (1, 1) strides and (1, 1) kernel_size


In [18]:
model.add(GlobalAveragePooling2D(name='global_average_pooling2d'))
model.add(Dense(units=4,activation='softmax'))

In [19]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_35 (Conv2D)              │ (None, 256, 256, 8)    │           216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_65          │ (None, 256, 256, 8)    │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 256, 256, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_30             │ (None, 256, 256, 8)    │            72 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_66          │ (None, 256, 256, 8)    │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 256, 256, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ (None, 256, 256, 16)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_67          │ (None, 256, 256, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 256, 256, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_31             │ (None, 127, 127, 16)   │           144 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_68          │ (None, 127, 127, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 127, 127, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_37 (Conv2D)              │ (None, 127, 127, 32)   │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_69          │ (None, 127, 127, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_32             │ (None, 127, 127, 32)   │           288 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_70          │ (None, 127, 127, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_38 (Conv2D)              │ (None, 127, 127, 32)   │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_71          │ (None, 127, 127, 32)   │           128 │
│ (BatchNormalization)            │                        │             

 Total params: 126,772 (495.20 KB)

 Trainable params: 123,860 (483.83 KB)

 Non-trainable params: 2,912 (11.38 KB)

In [43]:
model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])


In [44]:
model.fit(x=train_batches,validation_data=valid_batches,epochs=20,verbose=2)


Epoch 1/20
71/71 - 255s - 4s/step - accuracy: 0.9666 - loss: 0.1046 - val_accuracy: 0.9000 - val_loss: 0.3948
Epoch 2/20
71/71 - 232s - 3s/step - accuracy: 0.9652 - loss: 0.0944 - val_accuracy: 0.8417 - val_loss: 0.7309
Epoch 3/20
71/71 - 226s - 3s/step - accuracy: 0.9786 - loss: 0.0606 - val_accuracy: 0.9083 - val_loss: 0.4918
Epoch 4/20
71/71 - 235s - 3s/step - accuracy: 0.9777 - loss: 0.0706 - val_accuracy: 0.9000 - val_loss: 0.3574
Epoch 5/20
71/71 - 232s - 3s/step - accuracy: 0.9666 - loss: 0.0989 - val_accuracy: 0.8750 - val_loss: 0.6160
Epoch 6/20
71/71 - 231s - 3s/step - accuracy: 0.9795 - loss: 0.0649 - val_accuracy: 0.9250 - val_loss: 0.2289
Epoch 7/20
71/71 - 225s - 3s/step - accuracy: 0.9786 - loss: 0.0672 - val_accuracy: 0.9250 - val_loss: 0.2635
Epoch 8/20
71/71 - 227s - 3s/step - accuracy: 0.9889 - loss: 0.0434 - val_accuracy: 0.9083 - val_loss: 0.2273
Epoch 9/20
71/71 - 227s - 3s/step - accuracy: 0.9835 - loss: 0.0471 - val_accuracy: 0.9083 - val_loss: 0.2050
Epoch 10/2

In [45]:
predictions=model.predict(x=test_batches,verbose=0)
rounded_pred=np.round(predictions)

In [46]:
predict=np.argmax(rounded_pred,axis=1)


In [47]:
from sklearn.metrics import f1_score,accuracy_score
test_labels=test_batches.classes

f1_micro = f1_score(test_labels, predict,average='micro')
f1_micro*100

89.16666666666667

MobileNet v1 Lite with bitshift